In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from matplotlib import cm
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
#from visualization14 import ANN
#%matplotlib widget

In [2]:
# Synthetic dataset
from sklearn.datasets import make_classification
# Data processing
import pandas as pd
from collections import Counter
# Visualization
# Model and performance
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import copy

In [3]:
import pytorch_lightning as pl
from pytorch_lightning.core.decorators import auto_move_data
from torchvision.transforms import ToTensor
from torchvision.models import vgg16,mnasnet0_5
from torch import nn
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import Callback, ModelCheckpoint
from pytorch_lightning.metrics.functional import accuracy
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

In [4]:
BATCH_SIZE = 64
NUM_EPOCHS = 30
LEARNING_RATE = 0.005
NUM_WORKERS = 4

In [5]:
class PyTorchLeNet5(torch.nn.Module):

    def __init__(self, num_classes, grayscale=True):
        super().__init__()
        
        self.grayscale = grayscale
        self.num_classes = num_classes

        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3

        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, 6, kernel_size=5),
            torch.nn.Tanh(),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.Conv2d(6, 16, kernel_size=5),
            torch.nn.Tanh(),
            torch.nn.MaxPool2d(kernel_size=2)
        )

        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(16*5*5, 120),
            torch.nn.Tanh(),
            torch.nn.Linear(120, 84),
            torch.nn.Tanh(),
            torch.nn.Linear(84, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1)
        logits = self.classifier(x)
        return x,logits

In [6]:
import pytorch_lightning as pl
import torchmetrics


# LightningModule that receives a PyTorch model as input
class LightningModel(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()

        self.learning_rate = learning_rate
        # The inherited PyTorch module
        self.model = model

        # Save settings and hyperparameters to the log directory
        # but skip the model parameters
        self.save_hyperparameters()#(ignore=['model'])

        # Set up attributes for computing the accuracy
        self.train_acc = torchmetrics.Accuracy()
        self.valid_acc = torchmetrics.Accuracy()
        self.test_acc = torchmetrics.Accuracy()
        
    # Defining the forward method is only necessary 
    # if you want to use a Trainer's .predict() method (optional)
    def forward(self, x):
        return self.model(x)
        
    # A common forward step to compute the loss and labels
    # this is used for training, validation, and testing below
    def _shared_step(self, batch):
        features, true_labels = batch
        _,logits = self(features)
        loss = torch.nn.functional.cross_entropy(logits, true_labels)
        predicted_labels = torch.argmax(logits, dim=1)

        return loss, true_labels, predicted_labels

    def training_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.log("train_loss", loss)
        
        # To account for Dropout behavior during evaluation
        self.model.eval()
        with torch.no_grad():
            _, true_labels, predicted_labels = self._shared_step(batch)
        self.train_acc.update(predicted_labels, true_labels)
        self.log("train_acc", self.train_acc, on_epoch=True, on_step=False)
        self.model.train()
        return loss  # this is passed to the optimzer for training

    def validation_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.log("valid_loss", loss)
        self.valid_acc(predicted_labels, true_labels)
        self.log("valid_acc", self.valid_acc,
                 on_epoch=True, on_step=False, prog_bar=True)

    def test_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.test_acc(predicted_labels, true_labels)
        self.log("test_acc", self.test_acc, on_epoch=True, on_step=False)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [7]:
import os

from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader


class DataModule(pl.LightningDataModule):
    def __init__(self,train_data,valid_data):
        super().__init__()
        self.train_data=train_data
        self.valid_data=valid_data
        #self.data_path = data_path
        
    def prepare_data(self):
        #datasets.MNIST(root=self.data_path,
        #               download=True)
        self.resize_transform = transforms.Compose(
            [transforms.Resize((32, 32)),
             transforms.ToTensor()])
        
        return

    def setup(self, stage=None):
        # Note transforms.ToTensor() scales input images
        # to 0-1 range
        
        self.train, self.valid = self.train_data,self.valid_data

    def train_dataloader(self):
        print(len(self.train))
        train_loader = DataLoader(dataset=self.train, 
                                  batch_size=BATCH_SIZE, 
                                  drop_last=True,
                                  shuffle=True,
                                  num_workers=NUM_WORKERS)
        return train_loader

    def val_dataloader(self):
        valid_loader = DataLoader(dataset=self.valid, 
                                  batch_size=BATCH_SIZE, 
                                  drop_last=False,
                                  shuffle=False,
                                  num_workers=NUM_WORKERS)
        return valid_loader

In [8]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger


pytorch_model = PyTorchLeNet5(
    num_classes=10, grayscale=True)

lightning_model = LightningModel(
    model=pytorch_model, learning_rate=LEARNING_RATE)

callbacks = [ModelCheckpoint(
    save_top_k=1, mode='max', monitor="valid_acc")]  # save top 1 model 
logger = CSVLogger(save_dir="logs/", name="my-model")

In [9]:
transform = transforms.Compose(
            [transforms.Resize((32, 32)),
             transforms.ToTensor()])

In [10]:
MNIST('/.', train=True, download=True)
MNIST('/.', train=False, download=True)#data_dir
#transform = transforms.Compose([transforms.Grayscale(num_output_channels=3),transforms.ToTensor()])
dataset_full = MNIST('/.',train=True,transform=transform)
dataset_train = MNIST('/.',train=True,transform=transform)

#print(type(dataset_full),len(dataset_full)) 
dataset_train, dataset_val,remaining = random_split(dataset_full, [50000,10000,0])
#print(len(dataset_train),type(dataset_train))
dataset_train_original=copy.deepcopy(dataset_train)

In [11]:
torch.manual_seed(1) 
data_module = DataModule(dataset_train, dataset_val)

In [12]:
import time


trainer = pl.Trainer(
    max_epochs=NUM_EPOCHS,
    callbacks=callbacks,
    progress_bar_refresh_rate=50,  # recommended for notebooks
    #accelerator="auto",  # Uses GPUs or TPUs if available
    #devices="auto",  # Uses all available GPUs/TPUs if applicable
    logger=logger,
    log_every_n_steps=100)

start_time = time.time()
trainer.fit(model=lightning_model, datamodule=data_module)

runtime = (time.time() - start_time)/60
print(f"Training took {runtime:.2f} min in total.")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name      | Type          | Params
--------------------------------------------
0 | model     | PyTorchLeNet5 | 61.7 K
1 | train_acc | Accuracy      | 0     
2 | valid_acc | Accuracy      | 0     
3 | test_acc  | Accuracy      | 0     
--------------------------------------------
61.7 K    Trainable params
0         Non-trainable params
61.7 K    Total params
0.247     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

50000


C:\Users\USER\anaconda3\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: The ``compute`` method of metric Accuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Training took 90.69 min in total.


In [13]:
def prepare_test_embeddings(dataset_train):

    test_loader = DataLoader(dataset=dataset_train, 
                                     batch_size=BATCH_SIZE, 
                                     drop_last=False,
                                     shuffle=False,
                                     num_workers=NUM_WORKERS)
    test_dataloader = test_loader

    all_true_labels = []
    all_predicted_labels = []
    #test_imgs = torch.zeros((0, 1, 32, 32), dtype=torch.float32)
    test_embeddings = torch.zeros((0, 400), dtype=torch.float32)
    for batch in test_dataloader:
        features, labels = batch

        with torch.no_grad():  # since we don't need to backprop
            embeddings, logits = lightning_model(features)
        test_embeddings = torch.cat((test_embeddings, embeddings.detach().cpu()), 0)
        predicted_labels = torch.argmax(logits, dim=1)
        all_predicted_labels.append(predicted_labels)
        all_true_labels.append(labels)
        #test_imgs = torch.cat((test_imgs, features.detach().cpu()), 0)
    #test_imgs = np.array(test_imgs)

    test_embeddings = np.array(test_embeddings)
    all_predicted_labels = torch.cat(all_predicted_labels)
    all_true_labels = torch.cat(all_true_labels)
    all_predicted_labels=np.array(all_predicted_labels)
    all_true_labels=np.array(all_true_labels)
    
    
    accuracy = accuracy_score(all_predicted_labels, all_true_labels)#,average='micro')
    recall = recall_score(all_predicted_labels, all_true_labels,average='micro')
    precision = precision_score(all_predicted_labels, all_true_labels,average='micro')
    f1_score1 = f1_score(all_predicted_labels, all_true_labels,average='micro')
    
    print(f'Accuracy: {accuracy*100:.2f}%')
    print(f'precision: {precision*100:.2f}%')
    print(f'recall: {recall*100:.2f}%')
    print(f'f1_score1: {f1_score1*100:.2f}%')

    test_acc = np.sum(all_predicted_labels == all_true_labels)
    final_test_acc = test_acc/len(test_embeddings)
    #print(final_test_acc)
    #all_predicted_labels[:5]
    return test_embeddings,all_true_labels,all_predicted_labels,final_test_acc

## Shows some of MNIST data

In [14]:
import itertools
batch_size = 2
sample_at = 5
k = int(np.floor(sample_at/batch_size))

In [15]:
import itertools
k = int(np.floor(sample_at/batch_size))
#transform = transforms.Compose([transforms.Grayscale(num_output_channels=3),transforms.ToTensor()])
#data = MNIST(".", train=False, download=True, transform=transform)
data=copy.deepcopy(dataset_train)
dataloader = DataLoader(data, batch_size=32)
my_sample = next(itertools.islice(dataloader, k, None))
print (my_sample[1])

#plt.figure()
#for i in range(32):
#    plt.subplot(4,8,i+1)
#    arr=np.squeeze(data)
#    plt.imshow(my_sample[0][i][0], cmap='gray')
#    plt.axis('off')
#    plt.title(my_sample[1][i].item())

#dataloader = DataLoader(data, batch_size=32)

tensor([1, 0, 5, 5, 7, 0, 0, 4, 2, 3, 5, 9, 3, 7, 5, 2, 4, 2, 2, 6, 2, 3, 2, 4,
        3, 4, 9, 7, 5, 4, 8, 2])


## prepare_test_embeddings

In [16]:
# calculates precision for 1:100 dataset with 90 tp and 30 fp
from sklearn.metrics import precision_score,accuracy_score
# calculates recall for 1:100 dataset with 90 tp and 10 fn
from sklearn.metrics import recall_score
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score

In [17]:
data_test = MNIST(".", train=False, download=True, transform=transform)
_,_,_,final_test_acc=prepare_test_embeddings(data_test)
print('Initial Testing accuracy:  ',final_test_acc)

Accuracy: 97.92%
precision: 97.92%
recall: 97.92%
f1_score1: 97.92%
Initial Testing accuracy:   0.9792
